<a href="https://colab.research.google.com/github/blancavazquez/Taller_NLPWebminar/blob/main/notebook/training_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulación de la evolución clínica del paciente
## Blanca Hilda Vázquez Gómez

In [1]:
%load_ext tensorboard

## Paso 1: Carga de series de tiempo

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sn
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")  # avoid printing out absolute paths

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df_series_tiempo = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Taller_NLPWebminar/data/df_series_tiempo_webinar.csv")
df_demo = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Taller_NLPWebminar/data/df_demo_webinar.csv")

df_series_tiempo = df_series_tiempo.iloc[: , 1:]
df_demo = df_demo.iloc[: , 1:]

In [5]:
print("Tamaño de las series de tiempo:", df_series_tiempo.shape)
print("Tamaño de los datos demográficos:", df_demo.shape)

Tamaño de las series de tiempo: (165600, 15)
Tamaño de los datos demográficos: (3450, 6)


In [6]:
df_series_tiempo.head(5)

,Pid,time,BUN,Creatinine,HCT,HR,GCS,Temp,Platelets,HCO3,Na,WBC,K,Mg,Glucose
0,132539,0 days 00:07:00,NaN,NaN,NaN,75.0,15.0,35.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,132539,0 days 01:07:00,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,132539,0 days 02:07:00,NaN,NaN,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,132539,0 days 03:07:00,NaN,NaN,33.7,80.0,15.0,37.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,132539,0 days 04:07:00,NaN,NaN,NaN,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_demo.head(5)

,Pid,Mortalidad,Edad,TipoICU,Sexo,Altura
0,132539,0,54.0,4.0,0.0,NaN
1,132541,0,44.0,3.0,0.0,NaN
2,132548,0,68.0,3.0,0.0,162.6
3,132551,1,78.0,3.0,0.0,162.6
4,132555,0,74.0,2.0,1.0,175.3


### Generación de conjunto de entrenamiento y conjunto de prueba

In [8]:
"Ajustando el tamaño de los datos (cada fila = a las series de tiempo de un paciente)"
seqlength = 48
np_series_tiempo = df_series_tiempo.to_numpy().reshape(-1,seqlength*df_series_tiempo.shape[1])
print("Número de series de tiempo:",np_series_tiempo.shape)

Número de series de tiempo: (3450, 720)


In [9]:
"Creando conjuntos de entrenamiento (70%), validacion(20%), prueba (10%)"
x_train, x_test, y_train, y_test = train_test_split(np_series_tiempo, df_demo, test_size=0.1, random_state=422)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.22, random_state=422)

print("x_train:", x_train.shape,"y_train:", y_train.shape)
print("x_val:", x_val.shape, "y_val:", y_val.shape)
print("x_test:", x_test.shape, "y_test:", y_test.shape)

x_train: (2421, 720) y_train: (2421, 6)
x_val: (684, 720) y_val: (684, 6)
x_test: (345, 720) y_test: (345, 6)


In [10]:
"Separar las variables de [Pid, time] del resto de variables"
#feature_list = ['BUN', 'Creatinine', 'HCT', 'HR', 'GCS', 'Temp','Platelets', 'HCO3', 'Na', 'WBC', 'K', 'Mg', 'Glucose']
feature_list = ['HCT', 'HR', 'GCS', 'Temp']

#Conjunto de entrenamiento
x_train = x_train.reshape(-1*seqlength,df_series_tiempo.shape[1])
x_train = pd.DataFrame(x_train,columns = df_series_tiempo.columns)
x_train_pid = x_train[['Pid','time']]
x_train_data = x_train[feature_list]

#Conjunto de validación
x_val = x_val.reshape(-1*seqlength,df_series_tiempo.shape[1])
x_val = pd.DataFrame(x_val,columns = df_series_tiempo.columns)
x_val_pid = x_val[['Pid','time']]
x_val_data = x_val[feature_list]

#Conjunto de prueba
x_test = x_test.reshape(-1*seqlength,df_series_tiempo.shape[1])
x_test = pd.DataFrame(x_test,columns = df_series_tiempo.columns)
x_test_pid = x_test[['Pid','time']]
x_test_data = x_test[feature_list]

print("x_train: ",x_train.shape,"x_val: ",x_val.shape,"x_test:", x_test.shape)

x_train:  (116208, 15) x_val:  (32832, 15) x_test: (16560, 15)


In [11]:
print("x_train_pid:", x_train_pid.shape,"x_train_data:", x_train_data.shape)
print("x_val_pid:", x_val_pid.shape,"x_val_data:", x_val_data.shape)
print("x_test_pid:", x_test_pid.shape,"x_test_data:", x_test_data.shape)

x_train_pid: (116208, 2) x_train_data: (116208, 4)
x_val_pid: (32832, 2) x_val_data: (32832, 4)
x_test_pid: (16560, 2) x_test_data: (16560, 4)


A continuación vamos a definir un par de funciones.

- La primera función servirá para imputar los datos faltantes usando el imputador [iterativo](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html) de Scikit learn.
- La segunda función servirá para la escalar los datos, para esto usaremos [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Esta función transforma los datos de modo que su distribución tendrá una media = 0 y una desviación estándar de 1.

In [12]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

"Imputar los datos clínicos"
#imputer = IterativeImputer(imputation_order = 'ascending', n_nearest_features=5,random_state=242).fit(x_train_data)
imputer = SimpleImputer(strategy='mean').fit(x_train_data)

x_train_imp = np.around(imputer.transform(x_train_data),2)
x_val_imp = np.around(imputer.transform(x_val_data),2)
x_test_imp = np.around(imputer.transform(x_test_data),2)

print("x_train_imp:", x_train_imp.shape, "x_val_imp:", x_val_imp.shape, "x_test_imp:", x_test_imp.shape)

x_train_imp: (116208, 4) x_val_imp: (32832, 4) x_test_imp: (16560, 4)


In [13]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler #normalizar datos
"--- Normalizar los datos ---"
scaler = StandardScaler().fit(x_train_data)
#scaler = MinMaxScaler(feature_range=(-1,1)).fit(x_train_imp)
x_train_scaler = np.around(scaler.transform(x_train_imp),2)
x_val_scaler = np.around(scaler.transform(x_val_imp),2)
x_test_scaler = np.around(scaler.transform(x_test_imp),2)

print("x_train_scaler:", x_train_scaler.shape,"x_val_scaler:", x_val_scaler.shape,"x_test_scaler:",x_test_scaler.shape)

x_train_scaler: (116208, 4) x_val_scaler: (32832, 4) x_test_scaler: (16560, 4)


In [14]:
#Concatenar pids + data (imputados, escalados)
x_train_data = pd.DataFrame(x_train_scaler,columns = feature_list)
x_val_data = pd.DataFrame(x_val_scaler,columns = feature_list)
x_test_data = pd.DataFrame(x_test_scaler,columns = feature_list)

x_train_ = pd.concat([x_train_pid,x_train_data],axis=1)
x_val_ = pd.concat([x_val_pid,x_val_data],axis=1)
x_test_ = pd.concat([x_test_pid,x_test_data],axis=1)

print("x_train_:", x_train_.shape, "x_val_:", x_val_.shape,"x_test_:",x_test_.shape)

x_train_: (116208, 6) x_val_: (32832, 6) x_test_: (16560, 6)


## Construcción del modelo predictivo

El modelo usado en esta libreta está basado de la arquitectura de [Time series prediction using dilated causal convolutional neural nets](https://github.com/kristpapadopoulos/seriesnet)

In [15]:
!pip install pytorch-lightning #instalar pytorch lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import DataLoader
from pytorch_lightning import seed_everything, Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint,EarlyStopping

In [17]:
def split_df(df,in_seqlength,out_seqlength):
    """Split data: first hours (history), next hours (targets)"""
    start_index = 0
    end_index = in_seqlength+out_seqlength
    history = df[start_index:in_seqlength] #0-24 (first 24 hours)
    targets = df[in_seqlength:end_index] #(24-48) #next 24 hours
    return history, targets

def pad_arr(arr, expected_size):
    """ Pad top of array when there is not enough history """
    arr = np.pad(arr, [(expected_size - arr.shape[0], 0), (0, 0)], mode='edge')
    return arr

def df_to_np(df,seq_length):
    """ Convert dataframe to numpy """
    arr = np.array(df)
    arr = pad_arr(arr,seq_length)
    return arr

def rmse_loss(pred,true):
    "Computes the root mean square error"
    criterion = torch.nn.MSELoss()
    rmse = torch.sqrt(criterion(pred,true)) 
    return rmse

def mse_loss(pred,true):
    "Computes the mean square error"
    criterion = torch.nn.MSELoss()
    return criterion(pred,true)

def mae_loss(pred,true):
    "Computes the root mean absolute error (MAE)"
    criterion = torch.nn.L1Loss()
    mae = criterion(pred,true)
    return mae

def r2_corr(pred,true):
    return r2_score(true, pred).mean()

def rescale_data(np_data,seq_length,num_features):
    array_rescale= scaler.inverse_transform(np_data)
    array_rescale = np.around(array_rescale,2)
    array_rescale = array_rescale.reshape(-1,seq_length,num_features)
    return array_rescale

def plotting_predictions(real_input,real_target,pred,num_features,num_conditions,conditions,feature_list,mode,num_samples,epoch=0):
    seq_length = 24   
    real = np.array(real_input).reshape(-1*seq_length,num_features)
    target = np.array(real_target).reshape(-1*seq_length,num_features)
    prediction = np.array(pred).reshape(-1*seq_length,num_features)

    past_rescale = rescale_data(real,seq_length,num_features)
    expected_rescale = rescale_data(target,seq_length,num_features)
    pred_rescale = rescale_data(prediction,seq_length,num_features)

    # #selección de índices aleatorios from pred_seqs
    np.random.seed(42) 
    indices = np.random.choice(range(past_rescale.shape[0]),replace=False,size=num_samples)

    for index in indices: #recorriendo los indices
        past_pat = past_rescale[index, :, :] #(24, num_features)
        expected_pat = expected_rescale[index,:, :]
        prediction_pat= pred_rescale[index,:,:]

        pd_past_pat = pd.DataFrame(past_pat,columns = feature_list)
        pd_expected_pat = pd.DataFrame(expected_pat,columns = feature_list)
        pd_prediction_pat = pd.DataFrame(prediction_pat,columns = feature_list)

        if num_conditions > 0:
            np_cond = np.array(conditions).reshape(-1,num_conditions)
            conditions_per_patient = np_cond[index]

            age = conditions_per_patient[0]
            sex = conditions_per_patient[1].astype(int)

            # ----Converting One-hot to text
            sex_data = {'Sexo': [sex],}
            sex_pd = pd.DataFrame(sex_data)
            sex_pd['Sexo'] = np.where(sex_pd['Sexo'] == 1, 'Hombre','Mujer')

        fig = plt.figure(figsize=(20,15))
        plt.subplots_adjust(wspace=0.4,hspace=0.6)
        iplot = 420

        for i in range(len(feature_list)):
            feature = pd_prediction_pat.columns[i]
            iplot += 1
            if i == 12:
                ax = plt.subplot2grid((4,13), (i//4, 4), colspan=4)
            else:
                ax = plt.subplot2grid((4,4), (i//4,i%4))

            ax.plot(range(1,pd_past_pat.shape[0]+1),pd_past_pat[feature],label="Observaciones", color = "darkblue")
            ax.plot(range(pd_past_pat.shape[0], pd_past_pat.shape[0]+pd_expected_pat.shape[0]+1)
                    ,np.concatenate([pd_past_pat[feature][pd_past_pat.shape[0]-1:pd_past_pat.shape[0]],pd_expected_pat[feature]]),
                    label="Valor real",color = "orange")
            ax.plot(range(pd_past_pat.shape[0], pd_past_pat.shape[0]+pd_expected_pat.shape[0]+1)
                    ,np.concatenate([pd_past_pat[feature][pd_past_pat.shape[0]-1:pd_past_pat.shape[0]], pd_prediction_pat[feature]]),
                    label="Predicción",color='green')

            ax.set_title(feature)
            ax.grid(True,color='lightgrey',alpha=0.5)
            ax.set_xlabel("Tiempo (horas)")
            ax.set_ylabel("Valores")

        if num_conditions == 0:
          ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.35),fancybox=True, shadow=True, ncol=5)
        else:
          ax.legend(title='Sexo: '+str(sex_pd['Sexo'].item())+' Edad:'+str(age),
                      loc='upper center', bbox_to_anchor=(0.5, -0.35),fancybox=True, shadow=True, ncol=5)
        path = "/content/drive/MyDrive/Colab Notebooks/Taller_NLPWebminar/plots/"
        plt.savefig(path+str(mode)+"_epoch_"+str(epoch)+"_prediction_"+str(index)+".png")
        plt.close()

def weight_init(m):
    if isinstance(m, nn.Conv1d):
        torch.nn.init.trunc_normal_(m.weight, 0.0, 0.05)

In [18]:
num_vars = len(feature_list)

In [19]:
class CausalConv1d(torch.nn.Conv1d):
    def __init__(self,nb_filter,filter_length,dilation,device,target_length):
        self.__padding = ((filter_length - 1) * dilation)
       
        #(batch, in_channels, in_length) 
        super(CausalConv1d, self).__init__(num_vars,out_channels = nb_filter,kernel_size=filter_length,
            stride=1,padding=self.__padding,dilation=dilation,groups=1,bias=False,device=device)

    def forward(self, x_input,cond): 
        """ 
        Parameters:
        x_input = [batch_size,num_features,input_seqlength], 
        cond =[batch_size,num_conditions]

        Return:
        conv1d_out = [batch_size,32,input_seqlength]
        """
        out = x_input

        #[batch_size,num_features,input_seqlength] => [batch_size,32,input_seqlength]
        conv1d_out = super(CausalConv1d, self).forward(out)
        conv1d_out = conv1d_out[:, :, :-self.__padding] if self.__padding != 0 else conv1d_out
        return conv1d_out

class DC_CNN_Block(torch.nn.Module):
    def __init__(self,nb_filter, filter_length, dilation, num_features,target_length):
        super(DC_CNN_Block, self).__init__()
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.nb_filter = nb_filter
        self.filter_length= filter_length
        self.dilation = dilation
        self.num_features = num_features
        self.target_length = target_length
        
        #--- para condicion
        self.out_conv = nn.Conv1d(1,out_channels=self.target_length,kernel_size=1,stride=1,bias=False,device=device)
        self.out_linear = nn.Linear(in_features=770, out_features=32)
        self.out_conv.apply(weight_init)
        self.out_linear.apply(weight_init)
        #-----

        self.causal = CausalConv1d(self.nb_filter,self.filter_length,self.dilation,device=device,target_length=self.target_length)
        self.skip_out = nn.Conv1d(target_length,out_channels=self.num_features,kernel_size=1,stride=2,padding=8,bias=False,device=device)
        self.network_in = nn.Conv1d(target_length,out_channels=self.num_features,kernel_size=1,stride=2,padding=8,bias=False,device=device)
        
        # --- Initialize weights---
        self.causal.apply(weight_init) 
        self.skip_out.apply(weight_init)
        self.network_in.apply(weight_init)

    def forward(self, x_input,cond):
        dev = x_input.device

        residual = x_input #[32,num_features,seq_length]              
        x = x_input.permute(0,2,1).to(dev) #[32,seq_length,num_features]

        #[32,num_features,seq_length] => [32,32,seq_length]
        layer_out = self.causal(x,cond).to(dev)
        layer_out = layer_out.permute(0,2,1).to(dev) #[32, num_features, 32]
        layer_out = F.selu(layer_out) #[32, num_features, 32]

        #------------ Add conditions --------------------#
        if cond.shape[1] > 0:
          cond = cond.long()
          cond = cond.reshape(-1,cond.shape[1])#[32,num_cond]
          out = layer_out.reshape(-1,layer_out.shape[1]*layer_out.shape[2])
          x = torch.cat([out,cond],dim=1)
          x = x.reshape(-1,1,x.shape[1])
          out = self.out_conv(x)
          out = F.softmax(out)
          out = self.out_linear(out)
          layer_out = F.relu(out)
        # #------------------------------------------------#

        #[32, num_features, 32] => [32, num_features, seq_length]
        skip_out = self.skip_out(layer_out)
        skip_out = skip_out.permute(0,2,1).to(dev) #[32, seq_length, num_features]
        #[32, num_features, 32] => [32, num_features, seq_length]
        network_in = self.network_in(layer_out)
        network_in = network_in.permute(0,2,1).to(dev) #[32, seq_length, num_features]
        network_out = residual + network_in
        return network_out, skip_out

class CausalModel(pl.LightningModule):
    def __init__(self,target_length,num_features,num_conditions, feature_list,
                 in_channels,out_channels,kernel_size,stride,dilation,groups,bias):

        super(CausalModel,self).__init__()
        self.target_length = target_length
        self.num_features = num_features
        self.num_conditions = num_conditions
        self.feature_list = feature_list
        self.save_hyperparameters()

        self.block_1 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=1, num_features=self.num_features,target_length=self.target_length)
        self.block_2 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=2, num_features=self.num_features,target_length=self.target_length)
        self.block_3 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=4, num_features=self.num_features,target_length=self.target_length)
        self.block_4 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=8, num_features=self.num_features,target_length=self.target_length)
        self.block_5 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=16, num_features=self.num_features,target_length=self.target_length)
        self.block_6 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=32, num_features=self.num_features,target_length=self.target_length)
        self.block_7 = DC_CNN_Block(nb_filter=32, filter_length=2, dilation=64, num_features=self.num_features,target_length=self.target_length)
        self.l21 = nn.Conv1d(self.num_features,out_channels=self.num_features,kernel_size=1,stride=2,padding=12,bias=False)

        # Initialize weights 
        self.block_1.apply(weight_init)
        self.block_2.apply(weight_init)
        self.block_3.apply(weight_init)
        self.block_4.apply(weight_init)
        self.block_5.apply(weight_init)
        self.block_6.apply(weight_init)
        self.block_7.apply(weight_init)
        self.l21.apply(weight_init)

    def forward(self,x_input,cond): 
        """
        input sequence: [batch_size, target_length,num_features]
        condition: [batch_size,num_conditions]
        """
        dev = x_input.device #[32,seq_length,num_features]
        l1a, l1b = self.block_1(x_input,cond)
        l2a, l2b = self.block_2(l1a,cond)
        l3a, l3b = self.block_3(l2a,cond)
        l4a, l4b = self.block_4(l3a,cond)
        l5a, l5b = self.block_5(l4a,cond)
        l6a, l6b = self.block_6(l5a,cond)
        l6b = nn.Dropout(0.8)(l6b)
        l7a, l7b = self.block_7(l6a,cond)
        l7b = nn.Dropout(0.8)(l7b)
        l8 = l1b + l2b + l3b + l4b + l5b + l6b + l7b
        l9 = F.relu(l8)

        l9 = l9.permute(0,2,1).to(dev)
        l21 = self.l21(l9)
        l21 = l21.permute(0,2,1).to(dev)
        return l21

    def training_step(self, batch, batch_idx):
        target_in, target_out,condition = batch
        condition = condition[:,:self.num_conditions] #settings to number of conditions
        target_in = torch.tensor(target_in, dtype=torch.float32).to(target_in.device)
        target_out = torch.tensor(target_out, dtype=torch.float32).to(target_out.device)       
        
        y_pred = self(target_in,condition)#[32, seq_length,num_features]
        rmse = rmse_loss(y_pred,target_out)
        self.log("loss_train", rmse,on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {"loss":rmse,"past":target_in,"ytrue":target_out,"ypred":y_pred,"conditions":condition}

    def training_epoch_end(self, training_step_outputs):
        loss_training = torch.flatten(torch.stack([x['loss'] for x in training_step_outputs]))
        loss_training = loss_training.view(-1).detach().cpu().numpy().reshape(-1,1)

        # Convert from list to tensor
        past = torch.flatten(torch.stack([x['past'] for x in training_step_outputs]))
        ytrue = torch.flatten(torch.stack([x['ytrue'] for x in training_step_outputs]))
        ypred = torch.flatten(torch.stack([x['ypred'] for x in training_step_outputs]))
        conditions = torch.flatten(torch.stack([x['conditions'] for x in training_step_outputs]))

        past = past.view(-1).detach().cpu().numpy().reshape(-1,1)
        ytrue = ytrue.view(-1).detach().cpu().numpy().reshape(-1,1)
        ypred = ypred.view(-1).detach().cpu().numpy().reshape(-1,1)
        conditions = conditions.view(-1).detach().cpu().numpy().reshape(-1,1)

        plotting_predictions(past,ytrue,ypred,len(feature_list),self.num_conditions,conditions,feature_list,"train",1,self.current_epoch)

    def validation_step(self, batch, batch_idx):
        target_in, target_out,condition = batch
        condition = condition[:,:self.num_conditions] #settings to number of conditions
        target_in = torch.tensor(target_in, dtype=torch.float32).to(target_in.device)
        target_out = torch.tensor(target_out, dtype=torch.float32).to(target_out.device)

        y_pred = self(target_in,condition)#[32, seq_length,num_features]  
        rmse = rmse_loss(y_pred,target_out)
        self.log("loss_val",rmse)
        return {"loss_val":rmse,"past":target_in,"ytrue":target_out,"ypred":y_pred,"conditions":condition}

    def validation_epoch_end(self, validation_step_outputs):
        loss_val = torch.flatten(torch.stack([x['loss_val'] for x in validation_step_outputs]))
        loss_val = loss_val.view(-1).detach().cpu().numpy().reshape(-1,1)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),lr=0.001,betas=(0.9, 0.999),weight_decay=0.0)
        return {"optimizer": optimizer,"monitor": "loss",}   

class Dataset(torch.utils.data.Dataset):
    def __init__(self, groups, grp_by, target,demo,num_features):
        self.groups = groups
        self.grp_by = grp_by
        self.features = target
        self.demo = demo
        self.num_features = num_features
        self.seq_length = 24

    def __len__(self):
        return len(self.groups)

    def __getitem__(self, idx):
        """ targets_in = first 24 hours, targets_out = next 24 hours """
        list_targets_in=[]
        pid_patient = self.groups[idx]
        df = self.grp_by.get_group(pid_patient) #get all features for each patient
        df_conditions = self.demo.get_group(pid_patient)
        df_conditions = df_conditions[['Edad','Sexo']]

        #----- Get EHR real ------#
        #history=first24hrs, targets=next24hours
        history, targets = split_df(df,self.seq_length,self.seq_length)
        targets_in = history[self.features]
        targets_in = df_to_np(targets_in,self.seq_length)
        targets_in = targets_in.reshape(self.seq_length,self.num_features)
        targets_past = torch.tensor(targets_in, dtype=torch.float)

        targets_out = targets[self.features]
        targets_out = np.array(targets_out)
        targets_out = targets_out.reshape(self.seq_length,self.num_features)
        targets_expected = torch.tensor(targets_out, dtype=torch.float)

        # conditions
        conditions = np.around(np.array(df_conditions),1) #sex, age
        conditions = torch.tensor(conditions,dtype=torch.float) #[1,2]
        conditions = conditions.flatten()        
        return targets_past, targets_expected, conditions  

### Entrenamiento

In [ ]:
seed_everything(42) #for reproducibility
device = torch.device("cpu")

epochs = 500
batch_size = 32
seq_length = 24
num_condiciones = 2

x_train_ = x_train_[:115200] #115200 #4608
x_val_ = x_val_[:32256] #32256
x_test_ = x_test_[:15360] #15360

print("Epochs:",epochs,"Batch_size:",batch_size)
print("Demo data:", df_demo.shape)

print("x_train_:", x_train_.shape, "x_val_:", x_val_.shape,"x_test_:",x_test_.shape)

grp_by_train = x_train_.groupby(by=['Pid'])
grp_by_val = x_val_.groupby(by=['Pid'])
grp_by_test = x_test_.groupby(by=['Pid'])

groups_train = list(grp_by_train.groups)
groups_val= list(grp_by_val.groups)
groups_test= list(grp_by_test.groups)

print("groups_train:",len(groups_train))
print("groups_val:",len(groups_val))
print("groups_test:",len(groups_test))

#demographic data
grp_by_demo = df_demo.groupby(by=['Pid'])

full_groups_train = [grp for grp in groups_train if grp_by_train.get_group(grp).shape[0]>=2*seq_length]
full_groups_val = [grp for grp in groups_val if grp_by_val.get_group(grp).shape[0]>=2*seq_length]
full_groups_test = [grp for grp in groups_test if grp_by_test.get_group(grp).shape[0]>=2*seq_length]

print("full_groups_train:",len(full_groups_train))
print("full_groups_val:",len(full_groups_val))
print("full_groups_test:",len(full_groups_test))

train_data = Dataset(groups=full_groups_train,grp_by=grp_by_train,
                    target=feature_list,#son todas las vars, menos pids y offset
                    demo = grp_by_demo, #sex,gender
                    num_features = len(feature_list))

val_data = Dataset(groups=full_groups_val,grp_by=grp_by_val,
                   target=feature_list,#son todas las vars, menos pids y offset
                   demo = grp_by_demo, #sex,gender
                   num_features = len(feature_list))

test_data = Dataset(groups=full_groups_test,grp_by=grp_by_test,
                   target=feature_list,#son todas las vars, menos pids y offset
                   demo = grp_by_demo, #sex,gender
                   num_features = len(feature_list))

print("Train size (num patients):", len(train_data),"Val size (num patients):", len(val_data))

#Dataloader  = #num_samples/batch_size
train_loader = DataLoader(train_data,batch_size=batch_size,num_workers=1,shuffle=True) #num_workers=6
val_loader = DataLoader(val_data,batch_size=batch_size,num_workers=1,shuffle=False) #num_workers=5
test_loader = DataLoader(test_data,batch_size=batch_size,num_workers=1,shuffle=False) #num_workers=5

print("Train (loader):",len(train_loader),"Val (loader):",len(val_loader), "Test (loader):",len(test_loader))

model = CausalModel(target_length = seq_length,num_features = len(feature_list),
                    num_conditions = num_condiciones,feature_list = feature_list,
                    in_channels=32,out_channels=2,kernel_size=1,stride=1,dilation=1,groups=1,bias=False).to(device)

es_callback = EarlyStopping(patience=50, verbose=1, monitor='loss_val', mode='min')
checkpoint_callback = ModelCheckpoint(monitor='loss_val',mode="min",
                                      dirpath="/content/drive/MyDrive/Colab Notebooks/Taller_NLPWebminar/models/",
                                      filename='model_{epoch:02d}-{loss_val:.4f}',)

path = "/content/drive/MyDrive/Colab Notebooks/Taller_NLPWebminar/logs/"
trainer = Trainer(max_epochs = epochs,
                  gpus=1 if torch.cuda.is_available() else 0,
                  logger = TensorBoardLogger(path, name ='lightning_logs'),
                  callbacks=[checkpoint_callback,es_callback]) #es_callback

print("*** Starting training ***")
trainer.fit(model, train_loader, val_loader)

Global seed set to 42


Epochs: 500 Batch_size: 32
Demo data: (3450, 6)
x_train_: (115200, 6) x_val_: (32256, 6) x_test_: (15360, 6)
groups_train: 2400
groups_val: 672
groups_test: 320


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type         | Params
-----------------------------------------
0 | block_1 | DC_CNN_Block | 25.1 K
1 | block_2 | DC_CNN_Block | 25.1 K
2 | block_3 | DC_CNN_Block | 25.1 K
3 | block_4 | DC_CNN_Block | 25.1 K
4 | block_5 | DC_CNN_Block | 25.1 K
5 | block_6 | DC_CNN_Block | 25.1 K
6 | block_7 | DC_CNN_Block | 25.1 K
7 | l21     | Conv1d       | 16    
-----------------------------------------
176 K     Trainable params
0         Non-trainable params
176 K     Total params
0.704     Total estimated model params size (MB)


full_groups_train: 2400
full_groups_val: 672
full_groups_test: 320
Train size (num patients): 2400 Val size (num patients): 672
Train (loader): 75 Val (loader): 21 Test (loader): 10
*** Starting training ***


Sanity Checking: 0it [00:00, ?it/s]

...

Metric loss_val improved by 0.000 >= min_delta = 0.0. New best score: 0.624


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
!kill 1198

In [ ]:
# Start tensorboard
%tensorboard --logdir="/content/drive/MyDrive/Colab Notebooks/Taller_NLPWebminar/logs/lightning_logs"

### Rendimiento sobre el conjunto de prueba

In [ ]:
print("------------- Predictions on test set----------------")
list_rmse, list_r2 =[],[]
list_past,list_pred, list_real,list_cond = [],[],[],[]
seq_length = 24
model.eval()

with torch.no_grad(): #turn off gradients computation
    for i,data in enumerate(test_loader):
      real_input, real_target, conditions = data #[batch_size, seqlength, num_features]

      past = torch.tensor(real_input, dtype=torch.float32).to(device).reshape(-1,seq_length,len(feature_list)) #[32, 24, 13])
      expected = torch.tensor(real_target, dtype=torch.float32).to(device).reshape(-1,seq_length,len(feature_list)) #[32, 24, 13])

      #---- Prediction ----#
      model = model.to(device)
      pred = model(past,conditions)

      rmse = rmse_loss(pred,expected)
      score_R2 = r2_corr(pred.reshape(-1).detach().cpu().numpy().reshape(-1,1),expected.view(-1).detach().cpu().numpy().reshape(-1,1))

      list_rmse.append(rmse.item())
      list_r2.append(score_R2)

      list_past.extend(past.view(-1).detach().cpu().numpy().reshape(-1,1))
      list_real.extend(expected.view(-1).detach().cpu().numpy().reshape(-1,1))
      list_pred.extend(pred.reshape(-1).detach().cpu().numpy().reshape(-1,1))
      list_cond.extend(conditions.reshape(-1).detach().cpu().numpy().reshape(-1,1))

print("* Rendimiento del modelo sobre el conjunto de prueba")
print("RMSE:", round(np.mean(list_rmse),3),"+-",round(np.std(list_rmse),3))
print("R2:", round(np.mean(list_r2),3),"+-",round(np.std(list_r2),3))

# Saving metrics
list_rmse= np.array(list_rmse).reshape(-1,1)
np.savetxt("/content/drive/MyDrive/Colab Notebooks/Taller_NLPWebminar/metrics/"+"rmse_f_"+str(len(feature_list))+"_nc_"+str(num_condiciones)+".txt", list_rmse, delimiter=',')
list_r2= np.array(list_r2).reshape(-1,1)
np.savetxt("/content/drive/MyDrive/Colab Notebooks/Taller_NLPWebminar/metrics/"+"r2_f_"+str(len(feature_list))+"_nc_"+str(num_condiciones)+".txt", list_r2, delimiter=',')


print("* Visualizando algunas series de tiempo predichas")
plotting_predictions(list_past,list_real,list_pred,len(feature_list),num_condiciones,list_cond,feature_list,"test",10)